<a href="https://colab.research.google.com/github/vit0rz/Aspirador/blob/main/LIsta_N2_Inteligencia_Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:**\
**Matricula:**

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?

💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [1]:
import heapq

def dijkstra(grafo, origem):
    """
    Algoritmo de Dijkstra
    Retorna o menor custo (distâncias) e o caminho (predecessores)
    """

    dist = {no: float('inf') for no in grafo}
    parent = {no: None for no in grafo}


    dist[origem] = 0


    fila = [(0, origem)]


    while fila:

        custo_atual, atual = heapq.heappop(fila)


        if custo_atual > dist[atual]:
            continue


        for vizinho, peso in grafo[atual].items():
            novo_custo = dist[atual] + peso

            if novo_custo < dist[vizinho]:
                dist[vizinho] = novo_custo
                parent[vizinho] = atual
                heapq.heappush(fila, (novo_custo, vizinho))

    return dist, parent


def reconstruir_caminho(parent, destino):
    """
    Reconstrói o caminho mais curto até o destino
    usando o dicionário de predecessores (parent)
    """
    caminho = []
    atual = destino

    while atual is not None:
        caminho.append(atual)
        atual = parent[atual]

    caminho.reverse()
    return caminho


grafo = {
    'Hospital': {'A': 4, 'B': 2},
    'A': {'C': 3},
    'B': {'A': 1, 'C': 5, 'D': 8},
    'C': {'D': 2, 'Local': 4},
    'D': {'Local': 1},
    'Local': {}
}


distancias, predecessores = dijkstra(grafo, 'Hospital')


caminho = reconstruir_caminho(predecessores, 'Local')


print("🕒 Custo mínimo até o local:", distancias['Local'])
print("🗺️ Caminho mais rápido:", " → ".join(caminho))



 # Por que Dijkstra exige arestas não negativas?

 #O algoritmo de Dijkstra pressupõe que ao expandir o nó com menor distância atual, esse valor já é o menor possível (definitivo).
 #Se existissem arestas negativas, um caminho posterior poderia reduzir o custo total, invalidando essa suposição e tornando o resultado incorreto.

🕒 Custo mínimo até o local: 9
🗺️ Caminho mais rápido: Hospital → B → A → C → D → Local


**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?

2. Por que a heurística Manhattan é admissível nesse caso?

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [3]:
import heapq
import random


def gerar_grid(tamanho=20, obstaculos_percent=0.15):
    grid = []
    for _ in range(tamanho):
        linha = []
        for _ in range(tamanho):

            if random.random() < obstaculos_percent:
                linha.append(1)
            else:
                linha.append(0)
        grid.append(linha)
    return grid


def heuristic(a, b):
    """
    Heurística de Manhattan — soma das diferenças absolutas das coordenadas
    h(n) = |x1 - x2| + |y1 - y2|
    """
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)


def a_star(grid, start, goal, h):
    tamanho = len(grid)
    fila = [(0, start)]
    heapq.heapify(fila)


    g_score = {start: 0}

    parent = {start: None}

    while fila:
        _, atual = heapq.heappop(fila)


        if atual == goal:
            break

        x, y = atual
        vizinhos = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

        for nx, ny in vizinhos:
            if 0 <= nx < tamanho and 0 <= ny < tamanho and grid[nx][ny] == 0:
                novo_custo = g_score[atual] + 1
                if (nx, ny) not in g_score or novo_custo < g_score[(nx, ny)]:
                    g_score[(nx, ny)] = novo_custo
                    prioridade = novo_custo + h((nx, ny), goal)
                    heapq.heappush(fila, (prioridade, (nx, ny)))
                    parent[(nx, ny)] = atual

    return parent, g_score



def reconstruir_caminho(parent, start, goal):
    caminho = []
    atual = goal
    while atual is not None:
        caminho.append(atual)
        atual = parent.get(atual)
    caminho.reverse()
    return caminho if caminho[0] == start else []



grid = gerar_grid()
start = (0, 0)
goal = (19, 19)

parent, g_score = a_star(grid, start, goal, heuristic)
caminho = reconstruir_caminho(parent, start, goal)

print("Início:", start)
print(" Objetivo:", goal)
print(" Custo total:", g_score.get(goal, "Sem caminho encontrado"))
print(" Caminho encontrado:", caminho)


🏁 Início: (0, 0)
🎯 Objetivo: (19, 19)
📏 Custo total: 38
🗺️ Caminho encontrado: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (1, 14), (1, 15), (1, 16), (2, 16), (3, 16), (4, 16), (4, 17), (5, 17), (6, 17), (7, 17), (8, 17), (9, 17), (10, 17), (11, 17), (12, 17), (12, 18), (13, 18), (14, 18), (14, 19), (15, 19), (16, 19), (17, 19), (18, 19), (19, 19)]


**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?

In [ ]:

class No:
    def __init__(self, valor):
        self.valor = valor
        self.esquerda = None
        self.direita = None



def inserir(raiz, valor)
    if raiz is None:
        return No(valor)
    if valor < raiz.valor:
        raiz.esquerda = inserir(raiz.esquerda, valor)
    else:
        raiz.direita = inserir(raiz.direita, valor)
    return raiz



def in_order(raiz):
    if raiz:
        in_order(raiz.esquerda)
        print(raiz.valor, end=" ")
        in_order(raiz.direita)


def pre_order(raiz):
    if raiz:
        print(raiz.valor, end=" ")
        pre_order(raiz.esquerda)
        pre_order(raiz.direita)


def post_order(raiz):

    if raiz:
        post_order(raiz.esquerda)
        post_order(raiz.direita)
        print(raiz.valor, end=" ")



if __name__ == "__main__":

    valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]

    raiz = None
    for v in valore


**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

2. Diferencie corretude e otimalidade nos algoritmos estudados.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

In [ ]:
Quando não é vantajoso usar A*, mesmo tendo uma heurística?

O uso do A* pode ser desvantajoso quando o espaço de busca é muito grande e a heurística não é informativa o suficiente (ou seja, não aproxima bem o destino). Nesses casos, o A* acaba explorando quase tantos nós quanto o Dijkstra, mas com maior custo de processamento e uso de memória, já que mantém listas abertas e fechadas.
Em mapas muito extensos ou com pouca variação de pesos, o A* pode se tornar ineficiente, sendo melhor usar algoritmos mais simples, como o Dijkstra ou BFS.

Diferencie corretude e otimalidade nos algoritmos estudados.

A corretude garante que o algoritmo encontra uma solução válida, ou seja, chega ao destino respeitando as regras do problema.
Já a otimalidade assegura que essa solução é a melhor possível, como o menor custo ou menor distância.
Por exemplo, tanto o Dijkstra quanto o A* são corretos, mas só serão ótimos se todos os pesos forem não negativos (no Dijkstra) e se a heurística for admissível e consistente (no A*).

Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

Em-ordem: em um sistema de e-commerce, para listar produtos em ordem crescente de preço.

Pré-ordem: em um jogo de construção ou motor gráfico, ao clonar a hierarquia de objetos (pai → filhos).

Pós-ordem: em um sistema de gerenciamento de memória, para deletar estruturas complexas (liberando filhos antes dos pais).
Cada percurso é essencial para tarefas diferentes — ordenar, copiar ou liberar — conforme a forma como percorre a árvore.

Como heurísticas inconsistentes podem afetar o resultado do A*?

Uma heurística inconsistente (ou não monótona) pode subestimar o custo real de forma irregular, fazendo o A* reabrir nós já visitados ou até escolher caminhos incorretos temporariamente.
Isso aumenta o número de expansões, reduz a eficiência e pode comprometer a otimalidade se o algoritmo não for adaptado para lidar com reaberturas.
Em resumo, heurísticas inconsistentes tornam o A* menos previsível e mais custoso, podendo perder desempenho ou precisão em mapas e jogos complexos.

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.